In [1]:
from PIL import Image, ImageDraw
import dlib
import imageio.v2 as imageio
import os

In [5]:
def load_png(image_path):
    try:
        # Membaca gambar menggunakan imageio
        img_array = imageio.imread(image_path)
        
        # Mengembalikan array NumPy
        return img_array

    except Exception as e:
        print("Terjadi kesalahan:", e)
        return None

In [6]:
def crop_face_png(image_path, output_folder):
    # Memuat gambar
    img_array = load_png(image_path)

    if img_array is not None:
        # Menginisialisasi detector wajah
        face_detector = dlib.get_frontal_face_detector()

        # Mendeteksi wajah pada gambar
        faces = face_detector(img_array)

        # Menginisialisasi prediktor landmark wajah
        landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        for face in faces:
            # Mendapatkan landmark wajah
            landmarks = landmark_predictor(img_array, face)

            # Mendapatkan koordinat landmark untuk bagian wajah yang akan di-crop
            left_eye_x = landmarks.part(36).x
            left_eye_y = landmarks.part(37).y
            right_eye_x = landmarks.part(39).x
            right_eye_y = landmarks.part(41).y
            left_eyebrow_x = landmarks.part(17).x  # Koordinat landmark alis kiri
            left_eyebrow_y = landmarks.part(19).y
            right_eyebrow_x = landmarks.part(26).x  # Koordinat landmark alis kanan
            right_eyebrow_y = landmarks.part(24).y
            nose_x = landmarks.part(30).x
            nose_y = landmarks.part(30).y
            ear_left_x = landmarks.part(0).x
            ear_left_y = landmarks.part(0).y
            ear_right_x = landmarks.part(16).x
            ear_right_y = landmarks.part(16).y

            # Menentukan batas crop untuk wajah
            top_margin = int((left_eyebrow_y) * 0.1)  # 20% dari tinggi alis sebagai margin atas
            #left_bound = left_eyebrow_x
            #right_bound = right_eyebrow_x
            top_bound = left_eyebrow_y - top_margin
            bottom_bound = ear_left_y + (nose_y - ear_left_y)  # Menggunakan setengah hidung sebagai batas bawah

            # Mendapatkan batas gambar asli
            original_width, original_height = img_array.shape[1], img_array.shape[0]

            # Menentukan batas crop untuk wajah
            left_bound = 0  # Batas kiri tidak boleh kurang dari 0
            right_bound = original_width  # Batas kanan tidak boleh lebih dari lebar gambar

            # Crop bagian wajah dari gambar
            face_image = Image.fromarray(img_array[top_bound:bottom_bound, left_bound:right_bound])

            # Menampilkan gambar wajah yang di-crop (opsional)
            # face_image.show()

            # Simpan gambar wajah yang di-crop ke folder yang sesuai
            output_path = os.path.join(output_folder, os.path.basename(image_path))
            face_image.save(output_path)

In [ ]:
# Ganti path file dengan path gambar PNG yang ingin Anda muat
image_path = "/Users/bayu/Library/CloudStorage/OneDrive-PTCitilinkIndonesia/kuliah/tesis/image/CS229/CK+/happiness/S050_006_00000023.png"
rootFold = "/Volumes/DiskFile/Kuliah/bahan thesis/ImageCrop/allwidth"

# Melakukan crop pada bagian wajah dari gambar
crop_face_png(image_path, rootFold)

In [7]:
# Fungsi untuk melakukan iterasi melalui semua folder dan gambar dalamnya
def process_all_images(root_folder):
    for foldername, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            # Periksa apakah file memiliki ekstensi gambar yang diizinkan
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                # Buat path lengkap ke gambar
                image_path = os.path.join(foldername, filename)

                # Buat path folder output yang sesuai dengan folder asal
                output_folder = os.path.join("/Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/allwidth", os.path.relpath(foldername, root_folder))

                # Pastikan folder output sudah ada, jika belum, buat folder baru
                os.makedirs(output_folder, exist_ok=True)

                # Lakukan cropping pada gambar dan simpan hasil ke folder yang sesuai
                crop_face_png(image_path, output_folder)
                

In [8]:
# Ganti path folder utama yang berisi semua subfolder dan gambar
root_folder = "/Users/bayu/Library/CloudStorage/OneDrive-PTCitilinkIndonesia/kuliah/tesis/image/CS229/CK+"

# Panggil fungsi untuk memproses semua gambar dalam folder
process_all_images(root_folder)
